In [1]:
from gtfparse import read_gtf
import polars as pl
from intervaltree import Interval, IntervalTree
import pandas as pd
pd.set_option('display.max_rows', 60)
from collections import defaultdict

import os, sys

sys.path.append("./../")
print(sys.path)

from synonymes.mirnaID import miRNA, miRNASynonymeTYPE, miRNAPART
from Bio import SeqIO
from io import StringIO
import base64, gzip
import json

from Bio.Seq import Seq 


['/usr/lib64/python311.zip', '/usr/lib64/python3.11', '/usr/lib64/python3.11/lib-dynload', '', '/home/j/joppich/.local/lib/python3.11/site-packages', '/usr/lib64/python3.11/site-packages', '/usr/lib64/python3.11/_import_failed', '/usr/lib/python3.11/site-packages', './../']


In [3]:
all_fasta_seq = {}

with gzip.open("/mnt/extproj/projekte/textmining/genome_annotations/GRCm38.p6.genome.fa.gz", "rt") as fin:
    for seq_record in SeqIO.parse(fin, "fasta"):
        all_fasta_seq[seq_record.id] = str(seq_record.seq)

In [4]:
[x for x in all_fasta_seq][:10]

['chr1',
 'chr2',
 'chr3',
 'chr4',
 'chr5',
 'chr6',
 'chr7',
 'chr8',
 'chr9',
 'chr10']

In [5]:
df = read_gtf("/mnt/extproj/projekte/textmining/genome_annotations/gencode.vM25.annotation.gtf")

INFO:root:Extracted GTF attributes: ['gene_id', 'gene_type', 'gene_name', 'level', 'mgi_id', 'havana_gene', 'transcript_id', 'transcript_type', 'transcript_name', 'transcript_support_level', 'tag', 'havana_transcript', 'exon_number', 'exon_id', 'protein_id', 'ccdsid', 'ont']


In [6]:
df.head()

seqname,source,feature,start,end,score,strand,frame,gene_id,gene_type,gene_name,level,mgi_id,havana_gene,transcript_id,transcript_type,transcript_name,transcript_support_level,tag,havana_transcript,exon_number,exon_id,protein_id,ccdsid,ont
cat,cat,cat,i64,i64,f32,cat,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""chr1""","""HAVANA""","""gene""",3073253,3074322,null,"""+""",0,"""ENSMUSG00000102693.1""","""TEC""","""4933401J01Rik""","""2""","""MGI:1918292""","""OTTMUSG00000049935.1""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",""""""
"""chr1""","""HAVANA""","""transcript""",3073253,3074322,null,"""+""",0,"""ENSMUSG00000102693.1""","""TEC""","""4933401J01Rik""","""2""","""MGI:1918292""","""OTTMUSG00000049935.1""","""ENSMUST00000193812.1""","""TEC""","""4933401J01Rik-201""","""NA""","""basic""","""OTTMUST00000127109.1""","""""","""""","""""","""""",""""""
"""chr1""","""HAVANA""","""exon""",3073253,3074322,null,"""+""",0,"""ENSMUSG00000102693.1""","""TEC""","""4933401J01Rik""","""2""","""MGI:1918292""","""OTTMUSG00000049935.1""","""ENSMUST00000193812.1""","""TEC""","""4933401J01Rik-201""","""NA""","""basic""","""OTTMUST00000127109.1""","""1""","""ENSMUSE00001343744.1""","""""","""""",""""""
"""chr1""","""ENSEMBL""","""gene""",3102016,3102125,null,"""+""",0,"""ENSMUSG00000064842.1""","""snRNA""","""Gm26206""","""3""","""MGI:5455983""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""",""""""
"""chr1""","""ENSEMBL""","""transcript""",3102016,3102125,null,"""+""",0,"""ENSMUSG00000064842.1""","""snRNA""","""Gm26206""","""3""","""MGI:5455983""","""""","""ENSMUST00000082908.1""","""snRNA""","""Gm26206-201""","""NA""","""basic""","""""","""""","""""","""""","""""",""""""


In [7]:
pl.Config.set_tbl_rows(50)
df.filter(pl.col("gene_name") == "Ccl2").filter(pl.col("feature").is_in(["gene", "transcript", "exon"])).select(["seqname", "start", "end", "gene_id", "gene_type", "gene_name","feature"]).unique()

seqname,start,end,gene_id,gene_type,gene_name,feature
cat,i64,i64,str,str,str,cat
"""chr11""",82035571,82037453,"""ENSMUSG00000035385.5""","""protein_coding""","""Ccl2""","""transcript"""
"""chr11""",82035610,82037450,"""ENSMUSG00000035385.5""","""protein_coding""","""Ccl2""","""transcript"""
"""chr11""",82036483,82036600,"""ENSMUSG00000035385.5""","""protein_coding""","""Ccl2""","""exon"""
"""chr11""",82036483,82037450,"""ENSMUSG00000035385.5""","""protein_coding""","""Ccl2""","""exon"""
"""chr11""",82035571,82035737,"""ENSMUSG00000035385.5""","""protein_coding""","""Ccl2""","""exon"""
"""chr11""",82035571,82037453,"""ENSMUSG00000035385.5""","""protein_coding""","""Ccl2""","""gene"""
"""chr11""",82035610,82035737,"""ENSMUSG00000035385.5""","""protein_coding""","""Ccl2""","""exon"""
"""chr11""",82036926,82037453,"""ENSMUSG00000035385.5""","""protein_coding""","""Ccl2""","""exon"""


In [8]:
mirBSDF = pd.read_csv("../../../genome_annotations/Predicted_Target_Locations.default_predictions.mm10.bed", sep="\t",header=None)
mirBSDF.columns = ("chr", "start", "end", "gene_mir", "score", "strand", "start2", "end2", "score8", "score9", "score10", "score11")
mirBSDF.head(5)

,chr,start,end,gene_mir,score,strand,start2,end2,score8,score9,score10,score11
0,chr1,3214499,3214506,Xkr4:miR-155-5p,75,-,3214499,3214506,"30,144,255",1,7,0
1,chr1,3214511,3214518,Xkr4:miR-330-3p.2,95,-,3214511,3214518,"255,0,0",1,7,0
2,chr1,3214513,3214520,Xkr4:miR-29-3p,97,-,3214513,3214520,"255,0,0",1,7,0
3,chr1,3214882,3214889,Xkr4:miR-182-5p,92,-,3214882,3214889,"30,144,255",1,7,0
4,chr1,3214882,3214890,Xkr4:miR-96-5p,99,-,3214882,3214890,"128,0,128",1,8,0


In [9]:
mirFamilies = pd.read_csv("../../../genome_annotations/miR_Family_Info.txt", sep="\t")
mirFamilies = mirFamilies[mirFamilies["Species ID"].isin([10090])]
print(mirFamilies.head())

mirFamily2miRNA = defaultdict(set)

for ri, row in mirFamilies.iterrows():
    mirFamily2miRNA[ row["miR family"] ].add( row["MiRBase ID"] )

            miR family  Seed+m8  Species ID       MiRBase ID  \
33  let-7-3p/miR-98-3p  UAUACAA       10090  mmu-let-7a-1-3p   
34  let-7-3p/miR-98-3p  UAUACAA       10090    mmu-let-7b-3p   
35  let-7-3p/miR-98-3p  UAUACAA       10090  mmu-let-7c-2-3p   
36  let-7-3p/miR-98-3p  UAUACAA       10090  mmu-let-7f-1-3p   
37  let-7-3p/miR-98-3p  UAUACAA       10090    mmu-miR-98-3p   

           Mature sequence  Family Conservation? MiRBase Accession  
33  CUAUACAAUCUACUGUCUUUCC                    -1      MIMAT0004620  
34  CUAUACAACCUACUGCCUUCCC                    -1      MIMAT0004621  
35  CUAUACAAUCUACUGUCUUUCC                    -1      MIMAT0005439  
36  CUAUACAAUCUAUUGCCUUCCC                    -1      MIMAT0004623  
37  CUAUACAACUUACUACUUUCCU                    -1      MIMAT0017023  


In [10]:
gene2mir2locations = defaultdict(lambda : defaultdict(set))

for ri, row in mirBSDF.iterrows():
    loc = (row["chr"], row["start"], row["end"], row["strand"])
        
    gene = row["gene_mir"].split(":")[0]
    mirFamily = row["gene_mir"].split(":")[1]
    
    mirnas = mirFamily2miRNA[mirFamily]
        

    for mirna in mirnas:
        
        entryName = "-".join(mirna.split("-")[1:3]).replace("mir", "miR")
        
        gene2mir2locations[gene][entryName].add(loc)
        


In [11]:
gene2mir2locations["Ccl2"]

defaultdict(set,
            {'miR-6540': {('chr11', 82037195, 82037203, '+')},
             'miR-124': {('chr11', 82037195, 82037203, '+')},
             'miR-5624': {('chr11', 82037195, 82037203, '+')}})

In [12]:
ctable = {"gene": "blueviolet", "mirna": "purple", "*": "black" }

def rowToExonGTF(row, exonID, padding, startOffset = 0, genomeName=None):
    
    startOffset = int(startOffset)-padding
    
    geneID = row[3]
    geneName = row[5]
    geneType = row[4]
    
    if genomeName is None:
        genomeName = geneName
    
    geneType = "gene"
    
    entryStart = int(row[1])
    entryEnd = int(row[2])
        
    if row[7] == "gene":
        gtfEntry = (genomeName, "MXPLORE", row[7], entryStart-startOffset, entryEnd-startOffset, ".", row[6], ".", 'gene_id "{geneName}"; gene_type "{geneType}"; gene_name "{geneName}";'.format(geneName=geneName, geneType=geneType) )
    elif row[7] == "transcript":
        gtfEntry = (genomeName, "MXPLORE", row[7], entryStart-startOffset, entryEnd-startOffset, ".", row[6], ".", 'gene_id "{geneName}"; gene_type "{geneType}"; gene_name "{geneName}"; transcript_type "{geneType}"; transcript_name "{geneName}"; transcript_id "{geneName}";'.format(geneName=geneName, geneType=geneType) )
    elif row[7] == "mirna":
        geneType = "mirna"
        gtfEntry = (genomeName, "MXPLORE", row[7], entryStart-startOffset, entryEnd-startOffset, ".", row[6], ".", 'gene_id "{geneName}"; gene_type "{geneType}"; gene_name "{geneName}"; transcript_type "{geneType}"; transcript_name "{geneName}"; transcript_id "{geneName}"; exon_id "exon_{exonID}"; exon_number {exonID};'.format(geneName=geneName, geneType=geneType, exonID=exonID) )
    elif row[7] in ["exon", "CDS"]:
        gtfEntry = (genomeName, "MXPLORE", row[7], entryStart-startOffset, entryEnd-startOffset, ".", row[6], ".", 'gene_id "{geneName}"; gene_type "{geneType}"; gene_name "{geneName}"; transcript_type "{geneType}"; transcript_name "{geneName}"; transcript_id "{geneName}"; exon_id "exon_{exonID}"; exon_number {exonID};'.format(geneName=geneName, geneType=geneType, exonID=exonID) )
    
    gtfEntry = list(gtfEntry)
    gtfEntry.append(" ")
    
    return "\t".join([str(x) for x in gtfEntry])

def create_annotation_for_gene(selected_gene, padding=100):

    tree = IntervalTree()
    geneDF = df.filter(pl.col("gene_name") == selected_gene).filter(pl.col("feature").is_in(["gene", "exon"])).select(["seqname", "start", "end", "gene_id", "gene_type", "gene_name", "strand", "feature"]).unique()

    geneRow = None
    for row in geneDF.iter_rows():
        
        row = list(row)
        row[2] = row[2] + 1
        row = tuple(row)
        
        if row[-1] == "exon":
            tree.addi(row[1], row[2], row)
            
        elif row[-1] == "gene":
            geneRow = row
            
    def add_set_elem(inset, elem):
        inset.add(elem)
        return inset

    tree.merge_overlaps(data_reducer=lambda x,y: add_set_elem(x,y), data_initializer=set())

    gene_io = StringIO("")
    #print(rowToExonGTF(geneRow, None, startOffset = geneRow[1]), file=gene_io)
    #print(geneRow)

    txRow = list(geneRow)
    txRow[7] = "transcript"
    print(rowToExonGTF(txRow, None, padding=padding, startOffset = geneRow[1]), file=gene_io)

    treeIntervals = [x for x in tree]

    if geneRow[6] == "+":
        treeIntervals = sorted(treeIntervals, key=lambda x: x.begin)
    else:
        treeIntervals = sorted(treeIntervals, key=lambda x: x.end, reverse=True)

    for i, intv in enumerate(treeIntervals):
        dataElem = list(list(intv.data)[0]) 
        
        dataElem[1] = intv.begin
        dataElem[2] = intv.end
        
        print(rowToExonGTF(dataElem, i, padding=padding, startOffset = geneRow[1]), file=gene_io)

    gene_io.seek(0)
    geneStr = gene_io.read()


    geneData = base64.b64encode(gzip.compress(bytes(geneStr, "utf-8")))

    all_tracks = [
        {"name": "{} structure".format(selected_gene), "format": "gtf", "url": "data:application/gzip;base64,{}".format(geneData.decode()),"displayMode":"EXPANDED","type":"annotation","autoHeight":"true", "colorBy": "gene_type", "colorTable": ctable},
    ]


    gene_sequence = all_fasta_seq[geneRow[0]][ (geneRow[1]-padding):(geneRow[2]+padding) ]
    
    if geneRow[6] == "-":
        # complement sequence
        rawSeq = Seq(gene_sequence)
        newSeq = rawSeq.complement()
        gene_sequence = str(newSeq)
    
    genomeSeq = ">{}\n{}".format(selected_gene, gene_sequence)
    genomeData = base64.b64encode(gzip.compress(bytes(genomeSeq, "utf-8")))


    idxName = selected_gene
    idxLength = len(gene_sequence)
    idxOffset = len(selected_gene) + 2
    idxLineBases = len(gene_sequence)
    idxLineWidth = idxLineBases

    idxStr = "{}\t{}\t{}\t{}\t{}\n".format(idxName, idxLength, idxOffset, idxLineBases, idxLineWidth)
    idxData = base64.b64encode(gzip.compress(bytes(idxStr, "utf-8")))

    outputjson = {
        "geneIdEnsembl":selected_gene,
        "geneSymbolHgnc":selected_gene,
        "genome":
            {
                "id":selected_gene,
                "name":selected_gene,
                "fastaURL": "data:application/gzip;base64,{}".format(genomeData.decode()),
                "indexURL":"data:application/gzip;base64,{}".format(idxData.decode())
            },
        "tracks": all_tracks,
        "startOffset": geneRow[1]
    }

    return outputjson

def create_tracks_mirna(selected_gene, selected_mirna, startOffset, padding):   
    
    mirnaBindingSites = gene2mir2locations.get(selected_gene, {}).get(selected_mirna, []) #{('chr2', 136872166, 136872174, '-')}

    mirna_io = StringIO("")

    for i, bs in enumerate(mirnaBindingSites):
        mirna_data = (bs[0], bs[1], bs[2], selected_mirna, 'mirna', selected_mirna, bs[3], 'mirna')
        print(rowToExonGTF(mirna_data, i, padding=padding, startOffset = startOffset, genomeName=selected_gene), file=mirna_io)
        
    mirna_io.seek(0)
    mirStr = mirna_io.read()
    mirnaData = base64.b64encode(gzip.compress(bytes(mirStr, "utf-8")))
    
    mirTrack = {"name": "{} binding sites".format(selected_mirna), "format": "gtf", "url": "data:application/gzip;base64,{}".format(mirnaData.decode()),"displayMode":"EXPANDED","type":"annotation","autoHeight":"true", "colorBy": "gene_type", "colorTable": ctable}

    return mirTrack

In [13]:
allGenes = set(df["gene_name"].unique())
len(allGenes)

55291

In [14]:
padding=100

In [15]:
allGeneAnnots = {}

for i, gene in enumerate(allGenes):
    
    if i % 1000 == 0:
        print("gene index", i)
    geneAnnot = create_annotation_for_gene(gene, padding=padding)
    gene = gene.upper()
    allGeneAnnots[gene] = geneAnnot


gene index 0
gene index 1000
gene index 2000
gene index 3000
gene index 4000
gene index 5000
gene index 6000
gene index 7000
gene index 8000
gene index 9000
gene index 10000
gene index 11000
gene index 12000
gene index 13000
gene index 14000
gene index 15000
gene index 16000
gene index 17000
gene index 18000
gene index 19000
gene index 20000
gene index 21000
gene index 22000
gene index 23000
gene index 24000
gene index 25000
gene index 26000
gene index 27000
gene index 28000
gene index 29000
gene index 30000
gene index 31000
gene index 32000
gene index 33000
gene index 34000
gene index 35000
gene index 36000
gene index 37000
gene index 38000
gene index 39000
gene index 40000
gene index 41000
gene index 42000
gene index 43000
gene index 44000
gene index 45000
gene index 46000
gene index 47000
gene index 48000
gene index 49000
gene index 50000
gene index 51000
gene index 52000
gene index 53000
gene index 54000
gene index 55000


In [26]:
#allGeneAnnots = {x.upper(): allGeneAnnots[x] for x in allGeneAnnots}

In [34]:
import pickle
with open("mm10_gene_annotation.pickle", "wb") as fout:
    pickle.dump(allGeneAnnots, fout)

In [17]:
mirBSDF.head(5)

,chr,start,end,gene_mir,score,strand,start2,end2,score8,score9,score10,score11
0,chr1,3214499,3214506,Xkr4:miR-155-5p,75,-,3214499,3214506,"30,144,255",1,7,0
1,chr1,3214511,3214518,Xkr4:miR-330-3p.2,95,-,3214511,3214518,"255,0,0",1,7,0
2,chr1,3214513,3214520,Xkr4:miR-29-3p,97,-,3214513,3214520,"255,0,0",1,7,0
3,chr1,3214882,3214889,Xkr4:miR-182-5p,92,-,3214882,3214889,"30,144,255",1,7,0
4,chr1,3214882,3214890,Xkr4:miR-96-5p,99,-,3214882,3214890,"128,0,128",1,8,0


In [27]:
allMirnaAnnots = dict()

for ri, row in mirBSDF.iterrows():
    
    gene_mir = row["gene_mir"]
    gene, mir = gene_mir.split(":",1)
    
    mgene = gene.upper()
    
    for mir in gene2mir2locations[gene]:
        
        
        
        startOffset = allGeneAnnots.get(mgene, {}).get("startOffset", -1)
        
        if startOffset < 0:
            continue
        
        mirAnnot = create_tracks_mirna(gene, mir, startOffset, padding)
        
        if not gene in allMirnaAnnots:
            allMirnaAnnots[mgene] = dict()
        
        allMirnaAnnots[mgene][mir] = mirAnnot
        
        if ri == 0:
            print(mirAnnot)

{'name': 'miR-155 binding sites', 'format': 'gtf', 'url': 'data:application/gzip;base64,H4sIANzLYmYC/4vILjLh9I0I8PEPcuXMzSzKS+S0MLO04LQwNzDl1OPUBeL01LzU+MwUBaXczCBdQ1NTJWsFsFBJZUEqSBCoByaUl5ibiqyupCgxrzi5KLOgBE01kgQePaiWplbk54GFwAwDmEheaW5SapGCgTWnAhcA3wY2TM4AAAA=', 'displayMode': 'EXPANDED', 'type': 'annotation', 'autoHeight': 'true', 'colorBy': 'gene_type', 'colorTable': {'gene': 'blueviolet', 'mirna': 'purple', '*': 'black'}}
{'name': 'miR-330 binding sites', 'format': 'gtf', 'url': 'data:application/gzip;base64,H4sIANzLYmYC/4vILjLh9I0I8PEPcuXMzSzKS+S0MDc0ABHmnHqcukCcnpqXGp+ZoqCUmxmka2xsoGStABYqqSxIBQkC9cCE8hJzU5HVlRQl5hUnF2UWlKCpRpLAowfV0tSK/DywEJgBF8krzU1KLVIwsOZU4AIA7HCNcs4AAAA=', 'displayMode': 'EXPANDED', 'type': 'annotation', 'autoHeight': 'true', 'colorBy': 'gene_type', 'colorTable': {'gene': 'blueviolet', 'mirna': 'purple', '*': 'black'}}
{'name': 'miR-29c binding sites', 'format': 'gtf', 'url': 'data:application/gzip;base64,H4sIANzLYmYC/4vILjLh9I0I8PEPcuXMzSzKS+S0MDc0AhGWnH

In [33]:
import pickle
with open("mm10_mirna_annotation.pickle", "wb") as fout:
    pickle.dump(allMirnaAnnots, fout)

In [39]:
import pickle
with open("hg19_gene_annotation.pickle", "rb") as fout:
    allHGeneAnnots = pickle.load(fout)

In [40]:
with open("hg19_mirna_annotation.pickle", "rb") as fout:
    allHMirAnnots = pickle.load(fout)

In [46]:
[x for x in allHMirAnnots["CXCL12"]]

['miR-23a',
 'miR-130a',
 'miR-23b',
 'miR-23c',
 'miR-101',
 'miR-144',
 'miR-221',
 'miR-1928',
 'miR-222',
 'miR-19b',
 'miR-19a',
 'miR-200a',
 'miR-141',
 'miR-137',
 'miR-135a',
 'miR-135b']